# 推論予算の割り当ての最適化

このノートブックでは、品質、コスト、パフォーマンスのバランスをとるために、Claude 3.7 Sonnet の推論予算の割り当てを最適化する方法を探ります。前の 2 つのレッスンで学んだことを基に、次の実用的な戦略を立てます。

1. タスク要件に基づいて **推論予算を動的に割り当てる**
2. 予算サイズ、応答品質、コスト間の **トレードオフを視覚化する**
3. さまざまな種類のアプリケーション用に **再利用可能なパターンを作成する**

このレッスンの終わりまでに、さまざまなタスクに割り当てる適切な量の「思考力」を決定するための実用的なアプローチを習得できます。

## 前提条件

このノートブックは、レッスン 1 と 2 で説明した概念に基づいています。
- Claude 3.7 の拡張思考機能の理解
- Claude を呼び出すための Bedrock API の知識
- タスクの複雑さの分類フレームワーク

まず、環境を設定して必要なライブラリをインポートしましょう。

In [ ]:
# Import required libraries
import boto3
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown, HTML

# Import our utility functions from previous lessons
import claude_utils

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

# Configure pandas display options
pd.set_option('display.max_colwidth', None)

#### Bedrock クライアントを初期化します。

In [ ]:
# Set up the Bedrock clients using our utility module
REGION = 'us-west-2'  # Change to your preferred region
bedrock, bedrock_runtime = claude_utils.create_bedrock_clients(REGION)

# Claude 3.7 Sonnet model ID
CLAUDE_37_SONNET_MODEL_ID = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

# Verify model availability
claude_utils.verify_model_availability(bedrock, CLAUDE_37_SONNET_MODEL_ID)

## 要約: 推論予算の概念

最適化戦略に進む前に、Claude の推論予算について学んだことを簡単にまとめましょう。

- **推論予算** は、Claude の拡張思考プロセスに割り当てられたトークンの数です
- 最小推論予算は **1,024 トークン** です
- 予算が大きいほど、複雑な問題に対するより徹底した推論が可能になります
- 拡張思考には、出力トークンの一部としてコストがかかります (100 万トークンあたり 15 ドル)

推論予算は、計算タスクに CPU 時間を割り当てるようなものと考えてください。より複雑なタスクは、より大きな割り当てから恩恵を受けますが、ある時点を超えると収穫逓減が起こります。

このレッスンでは、さまざまなタイプのタスクの「スイート スポット」を見つけることに焦点を当て、パフォーマンスとコスト効率の両方を最適化します。

## 予算配分のトレードオフを理解する

動的な予算配分システムを実装する前に、関係する主要なトレードオフを理解しましょう。

1. **予算サイズと応答品質**: 予算が大きいほど、一般的に推論が徹底され、応答の質も向上する可能性がありますが、収穫逓減の法則があります

2. **予算サイズと応答時間**: 予算が大きいほど、クロードが考える時間が長くなるため、処理時間が長くなります

3. **予算サイズとコスト**: 予算が大きいほど、トークンの使用量が増え、コストも高くなります

これらのトレードオフにより、最適化の課題が生じます。つまり、特定のタスクの品質、速度、コストのバランスが取れた適切な予算を見つけることです。

中程度に複雑な問題でさまざまな推論予算をテストし、結果を測定して、これらのトレードオフを検討してみましょう。

#### 異なる予算規模のテスト

In [ ]:
def test_reasoning_budgets(prompt, budget_sizes=[1024, 2048, 4096, 8192, 16384], max_tokens=2000):
    """
    Test different reasoning budget sizes on the same prompt and collect performance metrics
    
    Args:
        prompt (str): The prompt to test
        budget_sizes (list): List of reasoning budget sizes to test
        max_tokens (int): Maximum tokens for responses
        
    Returns:
        pd.DataFrame: Performance metrics for each budget size
    """
    results = []
    
    print(f"Testing prompt: {prompt[:100]}..." if len(prompt) > 100 else f"Testing prompt: {prompt}")
    print(f"Testing {len(budget_sizes)} different reasoning budget sizes...\n")
    
    for budget in budget_sizes:
        print(f"Testing reasoning budget: {budget} tokens")
        
        start_time = time.time()
        response = claude_utils.invoke_claude(
            bedrock_runtime,
            prompt, 
            CLAUDE_37_SONNET_MODEL_ID, 
            enable_reasoning=True,
            reasoning_budget=budget,
            max_tokens=max_tokens
        )
        elapsed_time = time.time() - start_time
        
        # Extract metrics
        input_tokens = response.get('usage', {}).get('inputTokens', 0)
        output_tokens = response.get('usage', {}).get('outputTokens', 0)
        total_tokens = response.get('usage', {}).get('totalTokens', 0)
        
        # Calculate costs (approximate)
        input_cost = input_tokens * 0.000003  # $3 per million tokens
        output_cost = output_tokens * 0.000015  # $5 per million tokens
        total_cost = input_cost + output_cost
        
        # Calculate efficiency (tokens per second)
        efficiency = total_tokens / elapsed_time if elapsed_time > 0 else 0
        
        # Store results
        results.append({
            'budget': budget,
            'elapsed_time': elapsed_time,
            'input_tokens': input_tokens,
            'output_tokens': output_tokens,
            'total_tokens': total_tokens,
            'input_cost': input_cost,
            'output_cost': output_cost,
            'total_cost': total_cost,
            'efficiency': efficiency,
            'response': claude_utils.extract_response_content(response)
        })
        
        print(f"Completed in {elapsed_time:.2f}s, {total_tokens} tokens, ${total_cost:.6f}\n")
    
    return pd.DataFrame(results)

# Test with a moderately complex problem
test_prompt = """
A retailer sells two products, Product A and Product B. Product A costs $20 to make and sells for $50. 
Product B costs $10 to make and sells for $25. The retailer has $2000 available for production costs, 
and warehouse space for at most 250 units in total.

Due to customer demand, they need to produce at least twice as many units of Product B as Product A.

What is the optimal production plan to maximize profit?
"""

# Run the test
budget_test_results = test_reasoning_budgets(test_prompt)

# Display a subset of the results (excluding the actual responses to keep the output manageable)
display_cols = ['budget', 'elapsed_time', 'total_tokens', 'total_cost', 'efficiency']
display(budget_test_results[display_cols])

#### トレードオフの視覚化:

In [ ]:
def visualize_budget_tradeoffs(df):
    """
    Create visualizations to show the tradeoffs between different budget sizes
    
    Args:
        df (pd.DataFrame): DataFrame with test results
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Response Time vs. Budget Size
    axes[0, 0].plot(df['budget'], df['elapsed_time'], marker='o', linestyle='-', linewidth=2)
    axes[0, 0].set_title('Response Time vs. Budget Size', fontsize=14)
    axes[0, 0].set_xlabel('Reasoning Budget (tokens)', fontsize=12)
    axes[0, 0].set_ylabel('Response Time (seconds)', fontsize=12)
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Token Usage vs. Budget Size
    axes[0, 1].plot(df['budget'], df['total_tokens'], marker='o', linestyle='-', linewidth=2, color='green')
    axes[0, 1].set_title('Token Usage vs. Budget Size', fontsize=14)
    axes[0, 1].set_xlabel('Reasoning Budget (tokens)', fontsize=12)
    axes[0, 1].set_ylabel('Total Tokens Used', fontsize=12)
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Cost vs. Budget Size
    axes[1, 0].plot(df['budget'], df['total_cost'], marker='o', linestyle='-', linewidth=2, color='red')
    axes[1, 0].set_title('Cost vs. Budget Size', fontsize=14)
    axes[1, 0].set_xlabel('Reasoning Budget (tokens)', fontsize=12)
    axes[1, 0].set_ylabel('Total Cost ($)', fontsize=12)
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Efficiency vs. Budget Size
    axes[1, 1].plot(df['budget'], df['efficiency'], marker='o', linestyle='-', linewidth=2, color='purple')
    axes[1, 1].set_title('Efficiency vs. Budget Size', fontsize=14)
    axes[1, 1].set_xlabel('Reasoning Budget (tokens)', fontsize=12)
    axes[1, 1].set_ylabel('Efficiency (Tokens/Second)', fontsize=12)
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Return the budget size with the highest efficiency
    max_efficiency_idx = df['efficiency'].idxmax()
    max_efficiency_budget = df.iloc[max_efficiency_idx]['budget']
    
    print(f"The most efficient budget size is {max_efficiency_budget} tokens")
    print(f"This produced {df.iloc[max_efficiency_idx]['total_tokens']} tokens")
    print(f"In {df.iloc[max_efficiency_idx]['elapsed_time']:.2f} seconds")
    print(f"At a cost of ${df.iloc[max_efficiency_idx]['total_cost']:.6f}")

# Visualize the results
visualize_budget_tradeoffs(budget_test_results)

## 応答品質の分析

定量的な指標 (時間、トークン、コスト) は重要ですが、さまざまな予算サイズでの応答の **品質** も考慮する必要があります。最小予算 (1,024 トークン) での応答と、テストした最大予算での応答を比較してみましょう。

この定性的な分析により、より大きな推論予算を割り当てることで得られる実際の価値を理解することができます。

#### 応答品質の比較:

In [ ]:
# Display the response with the smallest budget
min_budget = min(budget_test_results['budget'])
min_budget_response = budget_test_results[budget_test_results['budget'] == min_budget]['response'].iloc[0]

# Display the response with the largest budget
max_budget = max(budget_test_results['budget'])
max_budget_response = budget_test_results[budget_test_results['budget'] == max_budget]['response'].iloc[0]

# Create a comparison display
print(f"Response with {min_budget} tokens budget:")
display(Markdown(min_budget_response))

print(f"\nResponse with {max_budget} tokens budget:")
display(Markdown(max_budget_response))

## 動的予算配分の実装

関連するトレードオフを理解したので、タスクの複雑さに基づいて推論予算を調整する動的予算配分戦略を実装しましょう。

私たちのアプローチは次のようになります:

1. タスクの複雑さを分類する
2. 複雑さに基づいて推論予算を割り当てる
3. 時間的制約とコスト制約に基づいて予算調整を適用する
4. 継続的な改善のためにパフォーマンスの記録を維持する

これにより、パフォーマンスとコストの考慮事項のバランスを取りながら、さまざまなタスクに適応できるシステムが作成されます。

## タスクの複雑さを使用して予算配分をガイドする

動的予算配分戦略の最初のステップは、タスクの複雑さを判断することです。すべてのタスクに固定予算を使用する代わりに、タスクの複雑さに基づいて推論予算を調整します。

### タスクの複雑さ分類子

次の機能を実行する `classify_task_complexity` 関数を実装します:

1. より小さく効率的なモデル (Claude 3.5 Haiku) を使用して、タスクをすばやく分類します
2. タスクを 4 つの複雑さのレベルに分類します:
- **シンプル**: 基本的な事実に関する質問、簡単な計算
- **中程度**: 複数ステップの推論、中程度の分析
- **複雑**: 詳細な分析、制約の問題、システム設計
- **非常に複雑**: 高度な数学的証明、多段階のシステム設計

この分類は、予算配分戦略の基礎となります。この分類ステップに小型モデルを使用すると、プロセス全体に Claude 3.7 Sonnet を使用するよりも高速でコスト効率も高くなります。

これをワークフローの「トリアージ」ステップと考えてください。病院が各患者の緊急度を迅速に判断してリソースを適切に割り当てる方法に似ています。

#### タスクの複雑さの分類

In [ ]:
def classify_task_complexity(prompt, model_id='anthropic.claude-3-5-haiku-20241022-v1:0'):
    """
    Use a more efficient model to classify the complexity of a task
    
    Args:
        prompt (str): The prompt to classify
        model_id (str): Model ID to use for classification
        
    Returns:
        str: Complexity category ('simple', 'medium', 'complex', 'very_complex')
    """
    system_prompt = [
        {"text": """You are a task complexity classifier. Classify the complexity of the given task 
                   into one of these categories: 'simple', 'medium', 'complex', or 'very_complex'.
                   
                   Examples:
                   - simple: Basic factual questions, straightforward calculations
                   - medium: Multi-step reasoning, moderate analysis
                   - complex: Detailed analysis, constraint problems, system design
                   - very_complex: Advanced mathematical proofs, multi-stage system design
                   
                   Respond with only the category name, nothing else."""}
    ]
    
    messages = [
        {
            "role": "user",
            "content": [{"text": f"Classify the task complexity: {prompt}"}]
        }
    ]
    
    try:
        response = bedrock_runtime.converse(
            modelId=model_id,
            messages=messages,
            system=system_prompt,
            inferenceConfig={
                "temperature": 0,
                "maxTokens": 10  # Only need a short response
            }
        )
        
        # Extract the classification
        result = None
        if response.get('output', {}).get('message', {}).get('content'):
            content_blocks = response['output']['message']['content']
            for block in content_blocks:
                if 'text' in block:
                    result = block['text'].strip().lower()
                    break
        
        # Ensure valid category
        valid_categories = ['simple', 'medium', 'complex', 'very_complex']
        if result not in valid_categories:
            result = 'medium'  # Default if unclear
        
        return result
        
    except Exception as e:
        print(f"Error classifying complexity: {e}")
        return "medium"  # Default if error

## 動的予算アロケータ

タスクの複雑さを分類できるようになったので、適切な推論予算を決定するための体系的な方法が必要です。`DynamicBudgetAllocator` クラスはこの機能を提供します。

### 設計哲学

動的予算アロケータは、次の原則に従います。

1. **複雑性主導**: 予算サイズを決定する主な要因はタスクの複雑さです

2. **コンテキスト認識**: 状況に基づいて割り当てを調整します (時間に敏感、コスト制約あり)

3. **適応型**: パフォーマンスを追跡して、時間の経過とともに割り当ての決定を改善します

4. **実用的**: 理論上の理想と現実世界の制約のバランスをとります

### 仕組み

アロケータは、各複雑性レベルに対してデフォルトの予算範囲を維持し、制約に基づいて調整を適用します。

- **時間に敏感** な状況では、より迅速な対応を優先するために予算を削減します
- **コスト制約** のシナリオでは、経費を制御するために予算を制限します
- **単純なタスク** では、拡張された思考を完全に回避します (標準モードを使用)

このアプローチにより、一貫した割り当てロジックを維持しながら、さまざまなシナリオを柔軟に処理できます。

このクラスを実装して、実際にどのように動作するかを見てみましょう。

In [ ]:
class DynamicBudgetAllocator:
    """
    Allocates reasoning budgets dynamically based on task complexity and constraints
    """
    def __init__(self):
        # Default budget ranges by complexity
        self.default_budgets = {
            'simple': 0,  # No extended thinking for simple tasks
            'medium': 2048,
            'complex': 4096,
            'very_complex': 8192
        }
        
        # Budget adjustments for time sensitivity
        self.time_sensitive_adjustments = {
            'simple': 0,
            'medium': 0,  # No extended thinking when time-sensitive
            'complex': -2048,  # Reduce budget for time-sensitive tasks
            'very_complex': -4096  # Significant reduction for time-sensitive tasks
        }
        
        # Performance tracking
        self.performance_history = {}
    
    def allocate_budget(self, prompt, time_sensitive=False, cost_constrained=False):
        """
        Allocate an appropriate reasoning budget for a task
        
        Args:
            prompt (str): The user prompt
            time_sensitive (bool): Whether the task is time-sensitive
            cost_constrained (bool): Whether to prioritize cost saving
            
        Returns:
            dict: Allocation decision including reasoning budget and strategy details
        """
        # Step 1: Classify task complexity
        complexity = classify_task_complexity(prompt)
        
        # Step 2: Get base budget for this complexity
        base_budget = self.default_budgets.get(complexity, 2048)
        
        # Step 3: Apply adjustments
        final_budget = base_budget
        
        # Apply time sensitivity adjustment
        if time_sensitive and complexity in self.time_sensitive_adjustments:
            final_budget += self.time_sensitive_adjustments[complexity]
        
        # Apply cost constraint adjustment (reduce by 50% if cost constrained)
        if cost_constrained and final_budget > 0:
            final_budget = max(1024, final_budget // 2)  # Minimum 1024 if using extended thinking
        
        # Step 4: Determine whether to use extended thinking
        use_extended_thinking = final_budget >= 1024
        
        # If not using extended thinking, set budget to 0
        if not use_extended_thinking:
            final_budget = 0
        
        # Step 5: Create allocation decision
        allocation = {
            'complexity': complexity,
            'use_extended_thinking': use_extended_thinking,
            'reasoning_budget': final_budget,
            'time_sensitive': time_sensitive,
            'cost_constrained': cost_constrained
        }
        
        return allocation
    
    def update_performance(self, allocation, elapsed_time, token_count, cost):
        """
        Update performance history for continuous learning
        
        Args:
            allocation (dict): The allocation decision
            elapsed_time (float): Time taken for response
            token_count (int): Total tokens used
            cost (float): Total cost
        """
        complexity = allocation['complexity']
        budget = allocation['reasoning_budget']
        
        if complexity not in self.performance_history:
            self.performance_history[complexity] = []
        
        self.performance_history[complexity].append({
            'budget': budget,
            'elapsed_time': elapsed_time,
            'token_count': token_count,
            'cost': cost,
            'timestamp': time.time()
        })

# Create an instance of our allocator
budget_allocator = DynamicBudgetAllocator()

In [ ]:
def test_dynamic_allocation(prompts, allocator):
    """
    Test our dynamic budget allocator on a set of prompts
    
    Args:
        prompts (dict): Dictionary of prompt labels to prompt text
        allocator (DynamicBudgetAllocator): The budget allocator
        
    Returns:
        pd.DataFrame: Results of the test
    """
    results = []
    
    for label, prompt in prompts.items():
        print(f"\nTesting prompt: {label}")
        print(f"Prompt: {prompt[:100]}..." if len(prompt) > 100 else f"Prompt: {prompt}")
        
        # Get allocation for standard mode (not time-sensitive)
        standard_allocation = allocator.allocate_budget(prompt, time_sensitive=False)
        print(f"Standard mode allocation: {standard_allocation}")
        
        # Get allocation for time-sensitive mode
        time_sensitive_allocation = allocator.allocate_budget(prompt, time_sensitive=True)
        print(f"Time-sensitive allocation: {time_sensitive_allocation}")
        
        # Execute with the standard allocation
        print(f"\nExecuting with standard allocation...")
        start_time = time.time()
        
        response = claude_utils.invoke_claude(
            bedrock_runtime,
            prompt,
            CLAUDE_37_SONNET_MODEL_ID,
            enable_reasoning=standard_allocation['use_extended_thinking'],
            reasoning_budget=standard_allocation['reasoning_budget'],
            max_tokens=1000
        )
        
        elapsed_time = time.time() - start_time
        
        # Calculate metrics
        input_tokens = response.get('usage', {}).get('inputTokens', 0)
        output_tokens = response.get('usage', {}).get('outputTokens', 0)
        total_tokens = response.get('usage', {}).get('totalTokens', 0)
        total_cost = (input_tokens * 0.000003) + (output_tokens * 0.000015)
        
        # Update allocator's performance history
        allocator.update_performance(
            standard_allocation,
            elapsed_time,
            total_tokens,
            total_cost
        )
        
        # Store result
        results.append({
            'Prompt': label,
            'Complexity': standard_allocation['complexity'],
            'Use_Extended_Thinking': standard_allocation['use_extended_thinking'],
            'Reasoning_Budget': standard_allocation['reasoning_budget'],
            'Time_Sensitive_Budget': time_sensitive_allocation['reasoning_budget'],
            'Elapsed_Time': elapsed_time,
            'Total_Tokens': total_tokens,
            'Total_Cost': total_cost
        })
        
        print(f"Completed in {elapsed_time:.2f}s, {total_tokens} tokens, ${total_cost:.6f}")
    
    return pd.DataFrame(results)

# Test prompts of varying complexity
test_prompts = {
    "Simple_Fact": "What is the capital of France?",
    
    "Medium_Math": "If a rectangle has a perimeter of 30 units and a width of 5 units, what is its area?",
    
    "Complex_Analysis": """
    Analyze the advantages and disadvantages of implementing a universal basic income 
    in a developed economy. Consider economic, social, and political perspectives.
    """,
    
    "Very_Complex_Design": """
    Design a system for urban traffic management that optimizes traffic flow, reduces congestion,
    minimizes emissions, and adapts to changing conditions. Include sensing, data processing,
    decision-making components, and how they would interact.
    """
}

# Run the test
allocation_test_results = test_dynamic_allocation(test_prompts, budget_allocator)

# Display the results
display(allocation_test_results)

## さまざまなシナリオに対する実用的な戦略

当社の実験と分析に基づいて、さまざまなシナリオに対する実用的な戦略をいくつか紹介します。

### 時間に敏感なアプリケーション
応答時間が重要なアプリケーション (カスタマー サービス チャットボット、リアルタイム アシスタントなど) の場合:

- 単純および中程度の複雑さのタスクには標準モード (拡張思考なし) を使用する
- 複雑なタスクには最小限の推論予算 (1,024～2,048 トークン) を使用する
- 2 段階のアプローチを検討する: 最初に迅速な応答を行い、次に要求された場合に詳細な分析を行う

### 深さが重要なアプリケーション
推論の品質と深さが優先されるアプリケーション (研究支援、複雑な分析など) の場合:

- 最も単純なタスクを除くすべてのタスクに拡張思考を使用する
- 複雑および非常に複雑なタスクには十分な推論予算 (4,096 トークン以上) を割り当てる
- 段階的な拡張を検討する: 中程度の予算から始めて、必要に応じて増やす

### コストに敏感なアプリケーション
予算が厳しいアプリケーションの場合制約:

- 複雑なタスクと非常に複雑なタスクにのみ拡張思考を使用する
- テストで特定された最も効率的なサイズに推論予算を制限する
- 推論コストの繰り返しを回避するために、共通の応答をキャッシュすることを検討する

### バランスのとれたアプローチ
すべての要素のバランスをとる汎用アプリケーションの場合:

- 複雑性に基づく予算編成で動的割り当てシステムを使用する
- パフォーマンス メトリックを時間の経過とともに追跡して、さまざまなタスク タイプに最適な予算を特定する
- ユーザーからのフィードバックとビジネス要件に基づいて割り当てを調整する

#### 動的割り当て戦略の視覚化

In [ ]:
def visualize_allocation_strategy():
    """
    Create a visualization of our dynamic allocation strategy
    """
    # Define complexity levels and scenarios
    complexities = ['Simple', 'Medium', 'Complex', 'Very Complex']
    scenarios = ['Standard', 'Time-Sensitive', 'Cost-Constrained']
    
    # Get budgets for each combination
    budgets = {}
    for scenario in scenarios:
        budgets[scenario] = []
        time_sensitive = scenario == 'Time-Sensitive'
        cost_constrained = scenario == 'Cost-Constrained'
        
        for complexity in complexities:
            # Create a sample prompt for each complexity
            if complexity == 'Simple':
                prompt = "What is the capital of France?"
            elif complexity == 'Medium':
                prompt = "Explain the greenhouse effect in simple terms."
            elif complexity == 'Complex':
                prompt = "Analyze the impacts of artificial intelligence on employment in the next decade."
            else:  # Very Complex
                prompt = "Design a system for managing autonomous vehicle traffic in a smart city."
            
            # Get allocation
            allocation = budget_allocator.allocate_budget(
                prompt, 
                time_sensitive=time_sensitive,
                cost_constrained=cost_constrained
            )
            
            budgets[scenario].append(allocation['reasoning_budget'])
    
    # Create the visualization
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Set up the bar positions
    bar_width = 0.25
    r1 = np.arange(len(complexities))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]
    
    # Create the bars
    ax.bar(r1, budgets['Standard'], width=bar_width, label='Standard', color='blue', alpha=0.7)
    ax.bar(r2, budgets['Time-Sensitive'], width=bar_width, label='Time-Sensitive', color='red', alpha=0.7)
    ax.bar(r3, budgets['Cost-Constrained'], width=bar_width, label='Cost-Constrained', color='green', alpha=0.7)
    
    # Add labels and legend
    ax.set_xlabel('Task Complexity', fontsize=14)
    ax.set_ylabel('Reasoning Budget (tokens)', fontsize=14)
    ax.set_title('Dynamic Budget Allocation Strategy', fontsize=16)
    ax.set_xticks([r + bar_width for r in range(len(complexities))])
    ax.set_xticklabels(complexities)
    ax.legend()
    
    # Add a horizontal line at 1024 tokens (minimum budget)
    ax.axhline(y=1024, color='gray', linestyle='--', alpha=0.7)
    ax.text(3.5, 1100, 'Minimum Budget (1,024 tokens)', fontsize=10)
    
    plt.tight_layout()
    plt.show()

# Visualize our allocation strategy
visualize_allocation_strategy()

## ケース スタディ: 複雑なタスクの予算最適化

学んだことを応用するために、複雑なタスク (気候変動緩和戦略の策定) の推論予算を最適化するケース スタディを見てみましょう。

このケース スタディでは、次の方法を説明します。

1. タスクの複雑さに基づいて適切な予算から開始する
2. 結果を特定の要件に照らして評価する
3. 必要に応じて調整して最適なバランスを見つける

理想的な予算は、特定の優先順位 (品質、速度、コスト) によって異なりますが、このプロセスは適切なバランスを見つけるための体系的なアプローチを提供します。

In [ ]:
def run_case_study():
    """
    Run a case study on budget optimization for a complex task
    """
    # Define our complex task
    case_study_prompt = """
    Develop a comprehensive strategy for a mid-sized city (population 500,000) to reduce its carbon emissions
    by 50% by 2035. Consider transportation, buildings, energy generation, industry, and waste management.
    Include specific policy recommendations, technological solutions, financing mechanisms, and implementation timeline.
    """
    
    print("Case Study: Carbon Emission Reduction Strategy")
    print("-" * 80)
    print(f"Prompt: {case_study_prompt}")
    
    # Step 1: Classify the task
    complexity = classify_task_complexity(case_study_prompt)
    print(f"\nTask classified as: {complexity}")
    
    # Step 2: Get the recommended budget from our allocator
    allocation = budget_allocator.allocate_budget(case_study_prompt)
    recommended_budget = allocation['reasoning_budget']
    print(f"Recommended budget: {recommended_budget} tokens")
    
    # Step 3: Test a range of budgets around the recommendation
    test_budgets = [
        max(1024, recommended_budget // 2),  # Half (or minimum)
        recommended_budget,                   # Recommended
        min(16384, recommended_budget * 2)    # Double (or maximum)
    ]
    
    print(f"\nTesting budgets: {test_budgets}")
    
    results = []
    response_texts = {}
    
    for budget in test_budgets:
        print(f"\nTesting budget: {budget} tokens")
        
        start_time = time.time()
        response = claude_utils.invoke_claude(
            bedrock_runtime,
            case_study_prompt,
            CLAUDE_37_SONNET_MODEL_ID,
            enable_reasoning=True,
            reasoning_budget=budget,
            max_tokens=1500
        )
        elapsed_time = time.time() - start_time
        
        # Extract metrics
        input_tokens = response.get('usage', {}).get('inputTokens', 0)
        output_tokens = response.get('usage', {}).get('outputTokens', 0)
        total_tokens = response.get('usage', {}).get('totalTokens', 0)
        total_cost = (input_tokens * 0.000003) + (output_tokens * 0.000015)
        
        # Store response text
        response_text = claude_utils.extract_response_content(response)
        response_texts[budget] = response_text
        
        # Calculate tokens per second
        tokens_per_second = total_tokens / elapsed_time if elapsed_time > 0 else 0
        
        # Store results
        results.append({
            'Budget': budget,
            'Time (s)': elapsed_time,
            'Tokens': total_tokens,
            'Cost ($)': total_cost,
            'Tokens/Second': tokens_per_second
        })
        
        print(f"Completed in {elapsed_time:.2f}s, {total_tokens} tokens, ${total_cost:.6f}")
    
    # Display results table
    results_df = pd.DataFrame(results)
    display(results_df)
    
    # Plot the results
    fig, ax = plt.subplots(1, 3, figsize=(18, 5))
    
    # Time plot
    ax[0].plot([r['Budget'] for r in results], [r['Time (s)'] for r in results], 'o-', linewidth=2)
    ax[0].set_title('Time vs. Budget')
    ax[0].set_xlabel('Budget (tokens)')
    ax[0].set_ylabel('Time (seconds)')
    ax[0].grid(True, alpha=0.3)
    
    # Cost plot
    ax[1].plot([r['Budget'] for r in results], [r['Cost ($)'] for r in results], 'o-', linewidth=2, color='red')
    ax[1].set_title('Cost vs. Budget')
    ax[1].set_xlabel('Budget (tokens)')
    ax[1].set_ylabel('Cost ($)')
    ax[1].grid(True, alpha=0.3)
    
    # Efficiency plot
    ax[2].plot([r['Budget'] for r in results], [r['Tokens/Second'] for r in results], 'o-', linewidth=2, color='green')
    ax[2].set_title('Efficiency vs. Budget')
    ax[2].set_xlabel('Budget (tokens)')
    ax[2].set_ylabel('Tokens per Second')
    ax[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Show response previews
    for budget, text in response_texts.items():
        print(f"\nResponse preview ({budget} tokens budget):")
        print("-" * 80)
        # Display first 300 characters
        preview = text[:300] + "..." if len(text) > 300 else text
        print(preview)
    
    # Final recommendation
    most_efficient_idx = np.argmax([r['Tokens/Second'] for r in results])
    most_efficient_budget = results[most_efficient_idx]['Budget']
    
    fastest_idx = np.argmin([r['Time (s)'] for r in results])
    fastest_budget = results[fastest_idx]['Budget']
    
    print("\nRecommendations:")
    print(f"- For maximum efficiency: {most_efficient_budget} tokens budget")
    print(f"- For fastest response: {fastest_budget} tokens budget")
    print("- For optimal quality/cost balance: Review the response content and choose based on your requirements")

# Run the case study
run_case_study()

## 結論とベスト プラクティス

このノートブックでは、Claude 3.7 Sonnet の推論予算の割り当てを最適化して、品質、コスト、パフォーマンスのバランスをとる方法を検討しました。主なポイントは次のとおりです:

### 学んだこと

1. **予算規模のトレードオフ**:
- 予算が大きいほど、一般的にはより綿密な推論が行われます
- ただし、応答時間とコストも増加します
- タスクの種類ごとに、収穫逓減のポイントがあります

2. **動的割り当て戦略**:
- タスクの複雑さは、予算割り当ての主な要因です
- 時間に対する敏感さとコストの制約は、重要な二次要因です
- シナリオによって、割り当て戦略は異なります

3. **最適な予算の見つけ方**:
- 複雑さに基づく推奨事項から始めます
- さまざまな予算をテストして、品質、時間、コストのバランスを見つけます
- パフォーマンス メトリックを追跡して、時間の経過とともにアプローチを改良します

### ベスト プラクティス

1. **本番システムの場合**:
- 最初のステップとしてタスクの複雑さの分類を実装します
- ユース ケースの要件に基づいて、動的な予算割り当てを使用します
- パフォーマンスを監視および分析して、継続的に最適化します

2. **コスト最適化の場合**:
- 拡張思考のみを使用します複雑さがそれを正当化する場合
- 各タスク タイプで「効率のスイート スポット」を見つける
- 一般的な応答をキャッシュすることを検討する

3. **品質の最適化について**:
- 品質が重要な複雑なタスクには、より大きな予算を割り当てる
- より大きな予算が必要な時期を識別するためのフィードバック ループを実装する
- 推論プロセスに焦点を合わせるようにプロンプ​​トを調整することを検討する

これらのプラクティスを実装することで、Claude 3.7 Sonnet の拡張された思考機能を最大限に活用しながら、特定の要件に合わせて最適化できます。